# Deep Learning Heston Model Calibration on Crypto-currencies Option's 

## Heston's Stochastic Volatility Model under real world probability measure

$\large dS_t = \mu S_t dt + \sqrt{v_t} S_t dW^\mathbb{P}_{1,t}$

$\large dv_t = \kappa (\theta - v_t)dt + \sigma \sqrt{v_t} dW^\mathbb{P}_{2,t}$

$\large \rho dt = dW^\mathbb{P}_{2,t} dW^\mathbb{P}_{2,t} $


Notation:
- $S_t$ Equity spot price, financial index
- $v_t$ Variance.
- $C$ European call option price.
- $K$ Strike price.
- $W_{1,2}$ Standard Brownian movements.
- $r$ Interest rate.
- $\kappa$ Mean reversion rate.
- $\theta$ Long run variance.
- $v_0$ Initial variance.
- $\sigma$ Volatility of variance.
- $\rho$ Correlation parameter.
- $t$ Current date.
- $T$ Maturity date.

In [209]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from datetime import datetime as dt
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn as nn
import torch.optim as optim
from scipy.stats import norm
from scipy.optimize import minimize, root
import QuantLib as ql

pd.set_option('display.max_columns', 100)

### Risk-free rate from US Daily Treasury Par Yield Curve Rates

**Cubic Spline approach**

In [210]:
from scipy.interpolate import CubicSpline

yield_maturities = np.array([1/12, 2/12, 3/12, 6/12, 1, 2, 3, 5, 7, 10, 20, 30])
yeilds = np.array([5.50, 5.47, 5.46, 5.41, 5.14, 4.83, 4.60, 4.44, 4.43, 4.42, 4.66, 4.56]).astype(float)/100

cs = CubicSpline(yield_maturities, yeilds)

# Interpolated values at specific points
x_new = np.linspace(1/12, 30, 100)
y_new = cs(x_new)

## Black Scholes Pricer

In [211]:
N = norm.cdf

def bs_call(S, K, T, r, sigma):
    if T <= 0 or sigma <= 0 or S <= 0 or K <= 0:
        return np.nan  # Return NaN for invalid inputs
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * N(d1) - K * np.exp(-r*T) * N(d2)

def bs_put(S, K, T, r, sigma):
    if T <= 0 or sigma <= 0 or S <= 0 or K <= 0:
        return np.nan  # Return NaN for invalid inputs
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return K * np.exp(-r*T) * N(-d2) - S * N(-d1)

## BTC and ETH Option Data DERIVIT API

#### Get Market Option Prices for BTC and ETH

In [212]:
# This is the script we ued for scfraping Deribit option data on btc and eth

df_btc = pd.read_csv("/Users/dnn/Option Pricing Model/data/btc_options.csv")
df_eth = pd.read_csv("/Users/dnn/Option Pricing Model/data/eth_options.csv")
df = df_btc

In [213]:
df["price"]= df["index_price"] * df["mark_price"]
df["mark_iv"]= df["mark_iv"]/100

# Convert maturity to numeric if it's in datetime format
df['maturity'] = pd.to_datetime(df['maturity'])
# Calculate the fraction of the year until maturity
df['year_fraction'] = (df['maturity'] - pd.Timestamp('today')).dt.days / 365
# Calculate the risk free rate for each maturity using the fitted yield curve
df['rf_rate'] = df['year_fraction'].apply(cs)

In [214]:
df['bs_price'] = df.apply(
    lambda x: bs_call(x['index_price'], x['strike'], x['year_fraction'], x['rf_rate'], x['mark_iv']) 
    if x['option_type'] == 'call' else 
    bs_put(x['index_price'], x['strike'], x['year_fraction'], x['rf_rate'], x['mark_iv']),
    axis=1)
df = df.dropna()
df[250:]

,option_type,strike,creation,maturity,mark_price,last_price,mark_iv,index_price,price,year_fraction,rf_rate,bs_price
313,call,56000.0,2024-04-24 08:01:00,2024-07-26 08:00:00,0.2416,0.1820,0.5606,68821.59,16627.296144,0.161644,0.054710,14575.674709
314,put,56000.0,2024-04-24 08:01:00,2024-07-26 08:00:00,0.0135,0.0195,0.5606,68821.59,929.091465,0.161644,0.054710,1261.029610
315,call,57000.0,2024-04-24 08:01:00,2024-07-26 08:00:00,0.2301,0.1620,0.5598,68821.73,15835.880073,0.161644,0.054710,13788.691448
316,put,57000.0,2024-04-24 08:01:00,2024-07-26 08:00:00,0.0157,0.0240,0.5598,68821.73,1080.501161,0.161644,0.054710,1465.101794
317,call,58000.0,2024-04-24 08:01:00,2024-07-26 08:00:00,0.2186,0.1520,0.5562,68821.73,15044.430178,0.161644,0.054710,13003.427235
...,...,...,...,...,...,...,...,...,...,...,...,...
573,call,170000.0,2024-03-27 08:01:00,2025-03-28 08:00:00,0.0590,0.1205,0.7367,68811.08,4059.853720,0.832877,0.052375,3248.424796
574,call,180000.0,2024-03-27 08:01:00,2025-03-28 08:00:00,0.0520,0.0625,0.7456,68811.07,3578.175640,0.832877,0.052375,2916.167646
575,put,180000.0,2024-03-27 08:01:00,2025-03-28 08:00:00,1.3876,2.0170,0.7456,68811.07,95482.240732,0.832877,0.052375,106421.905053
576,call,200000.0,2024-03-27 13:12:00,2025-03-28 08:00:00,0.0425,0.0470,0.7601,68811.06,2924.470050,0.832877,0.052375,2364.132402


In [215]:
# Define the strike range based on observed data density
'''min_strike = 2500
max_strike = 6000'''

min_strike = 55000
max_strike = 80000

df = df[(df['strike'] >= min_strike) & (df['strike'] <= max_strike) & 
                 (df['year_fraction'] > 0.02) & (df['year_fraction'] < 1)]

In [216]:
df = df[['option_type','index_price','mark_iv','year_fraction','strike','maturity','rf_rate','price']]
df

,option_type,index_price,mark_iv,year_fraction,strike,maturity,rf_rate,price
156,put,68793.39,0.5938,0.027397,56000.0,2024-06-07 08:00:00,0.055431,55.034712
157,call,68793.39,0.5529,0.027397,58000.0,2024-06-07 08:00:00,0.055431,12424.086234
158,put,68793.39,0.5529,0.027397,58000.0,2024-06-07 08:00:00,0.055431,82.552068
159,call,68793.48,0.5214,0.027397,60000.0,2024-06-07 08:00:00,0.055431,10532.281788
160,put,68793.48,0.5214,0.027397,60000.0,2024-06-07 08:00:00,0.055431,137.586960
...,...,...,...,...,...,...,...,...
550,put,68810.70,0.6675,0.832877,70000.0,2025-03-28 08:00:00,0.052375,12833.195550
551,call,68811.07,0.6782,0.832877,75000.0,2025-03-28 08:00:00,0.052375,17526.179529
552,put,68811.07,0.6782,0.832877,75000.0,2025-03-28 08:00:00,0.052375,15682.042853
553,call,68811.26,0.6818,0.832877,80000.0,2025-03-28 08:00:00,0.052375,15950.450068


In [217]:
# Assuming your dataframe df is already defined and contains the columns specified:
def augment_dataset(df, n_samples=1000):
    augmented_data = []
    for _ in range(n_samples):
        for _, row in df.iterrows():
            # Variations for mark_iv, strike, and year_fraction
            var_sigma = row['mark_iv'] * np.random.uniform(0.9, 1.1)
            var_strike = row['strike'] * np.random.uniform(0.9, 1.1)
            var_time = row['year_fraction'] * np.random.uniform(0.9, 1.1)
            
            # Generate new price based on varied parameters
            if row['option_type'] == 'call':
                new_price = bs_call(row['index_price'], var_strike, var_time, row['rf_rate'], var_sigma)
            else:
                new_price = bs_put(row['index_price'], var_strike, var_time, row['rf_rate'], var_sigma)


            # Create a new row with the modified parameters
            new_row = row.copy()
            new_row['mark_iv'] = var_sigma
            new_row['strike'] = var_strike
            new_row['year_fraction'] = var_time
            new_row['price'] = new_price
            
            augmented_data.append(new_row)
    
    # Convert the list of dictionaries to a DataFrame
    return pd.DataFrame(augmented_data)



In [218]:
augment_dataset = augment_dataset(df)

In [219]:
# Example usage:
df = augment_dataset
df

,option_type,index_price,mark_iv,year_fraction,strike,maturity,rf_rate,price
156,put,68793.39,0.649871,0.029196,56000.377946,2024-06-07 08:00:00,0.055431,82.707777
157,call,68793.39,0.507116,0.027416,55893.129779,2024-06-07 08:00:00,0.055431,12995.840936
158,put,68793.39,0.520314,0.025783,60314.800735,2024-06-07 08:00:00,0.055431,127.455869
159,call,68793.48,0.553682,0.029091,64758.390815,2024-06-07 08:00:00,0.055431,5105.860015
160,put,68793.48,0.572808,0.027986,62568.020531,2024-06-07 08:00:00,0.055431,516.996329
...,...,...,...,...,...,...,...,...
550,put,68810.70,0.732844,0.800853,72739.044837,2025-03-28 08:00:00,0.052375,18280.619610
551,call,68811.07,0.656987,0.908328,74424.673294,2025-03-28 08:00:00,0.052375,16119.293873
552,put,68811.07,0.734036,0.786738,77992.349008,2025-03-28 08:00:00,0.052375,21495.807228
553,call,68811.26,0.721930,0.869169,80788.337843,2025-03-28 08:00:00,0.052375,15316.071622


In [220]:
from sklearn.base import BaseEstimator, RegressorMixin
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import legacy as legacy_optimizers  # Use legacy optimizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

In [221]:
# Define the custom Keras regressor wrapper
class KerasRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, build_fn, epochs=100, batch_size=10, verbose=0):
        self.build_fn = build_fn
        self.epochs = epochs
        self.batch_size = batch_size
        self.verbose = verbose
        self.model = None

    def fit(self, X, y, **kwargs):
        self.model = self.build_fn(input_dim=X.shape[1])
        self.model.fit(X, y, epochs=self.epochs, batch_size=self.batch_size, verbose=self.verbose, **kwargs)
        return self

    def predict(self, X, **kwargs):
        return self.model.predict(X, **kwargs)



In [222]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation, LSTM
from tensorflow.keras.optimizers import Adam

def build_complex_model(input_dim):
    model = Sequential([
        Dense(128, input_shape=(input_dim,)),
        BatchNormalization(),  # Helps to normalize the inputs of each layer
        Activation('relu'),
        Dropout(0.3),  # Increased dropout rate for regularization
        
        Dense(128),
        BatchNormalization(),
        Activation('relu'),
        Dropout(0.3),
        
        Dense(64),
        BatchNormalization(),
        Activation('relu'),
        Dropout(0.2),

        Dense(32),
        BatchNormalization(),
        Activation('relu'),
        Dropout(0.1),

        Dense(1)  # Output layer
    ])
    
    model.compile(optimizer=legacy_optimizers.Adam(learning_rate=0.001), loss='mean_squared_error')
    return model

In [223]:
def build_lstm_model(input_shape):
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        
        LSTM(50, return_sequences=False),
        Dropout(0.2),
        
        Dense(25, activation='relu'),
        Dropout(0.1),

        Dense(1)
    ])
    
    model.compile(optimizer=legacy_optimizers.Adam(learning_rate=0.001), loss='mean_squared_error')
    return model 

In [224]:
# Define preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['index_price', 'mark_iv', 'year_fraction', 'strike', 'rf_rate']),
        ('cat', OneHotEncoder(), ['option_type'])
    ])

# Splitting data
features = df.drop('price', axis=1)
target = df['price']
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)


In [225]:
# Define the full pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', KerasRegressor(build_fn=build_complex_model, epochs=4, verbose=1))  # Note the epochs and verbose level
])

# Now fit the pipeline, not just the model
pipeline.fit(X_train, y_train)

Epoch 1/4
14720/14720 [==============================] - 140s 9ms/step - loss: 38278068.0000
Epoch 2/4
14720/14720 [==============================] - 138s 9ms/step - loss: 5271536.0000
Epoch 3/4
14720/14720 [==============================] - 756s 51ms/step - loss: 3271382.0000
Epoch 4/4
14720/14720 [==============================] - 139s 9ms/step - loss: 2584825.2500


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['index_price', 'mark_iv',
                                                   'year_fraction', 'strike',
                                                   'rf_rate']),
                                                 ('cat', OneHotEncoder(),
                                                  ['option_type'])])),
                ('regressor',
                 KerasRegressor(build_fn=<function build_complex_model at 0x32481eac0>,
                                epochs=4, verbose=1))])

In [226]:
# Predict and evaluate
y_pred = pipeline.predict(X_test)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"R^2 Score: {r2:.2f}")
print(f"RMSE: {rmse:.2f}")

1150/1150 [==============================] - 3s 3ms/step
R^2 Score: 0.97
RMSE: 991.77


In [241]:
def create_comparison_table(y_test, y_pred):
    # Create a DataFrame from the actual and predicted values
    df = pd.DataFrame({
        'True Price': y_test,
        'Predicted Price': y_pred.flatten()  # Flatten y_pred to ensure it matches the shape of y_test
    })
    df['Absolute Error'] = np.abs(df['True Price'] - df['Predicted Price'])
    df['Squared Error'] = (df['True Price'] - df['Predicted Price']) ** 2
    df['Error Percentage (%)'] = df['Absolute Error'] / df['True Price'] * 100

    return df[['True Price','Predicted Price','Error Percentage (%)' ]]

# Assuming y_test and y_pred are available from your previous model output
comparison_table = create_comparison_table(y_test, y_pred)
comparison_table = comparison_table[(comparison_table['Predicted Price'] > 0) & (comparison_table['Error Percentage (%)'] < 15)]
comparison_table 

,True Price,Predicted Price,Error Percentage (%)
311,12363.121180,11958.227539,3.275012
159,8929.889156,9226.750977,3.324362
433,11233.020318,12266.680664,9.201981
319,8859.566310,9271.804688,4.653031
504,16411.229944,15544.561523,5.280947
...,...,...,...
360,10179.868537,10707.724609,5.185294
422,18000.916770,17269.623047,4.062536
436,11959.052776,12147.405273,1.574978
554,20634.866884,18442.365234,10.625228


In [242]:
def generate_error_report_from_table(comparison_table):
    # Extract actual and predicted values from the DataFrame
    y_true = comparison_table['True Price'].values
    y_pred = comparison_table['Predicted Price'].values

    # Initialize sums and count for averaging
    mse_sum_error = 0.0
    mae_sum_error = 0.0
    sum_actual_price = 0.0
    count = 0.0

    for actual, predicted in zip(y_true, y_pred):
        mse_error = (predicted - actual) ** 2
        mae_error = abs(predicted - actual)

        # Avoid division by zero; add a small number to the denominator if actual is zero
        actual_nonzero = actual if actual != 0 else 1e-10  # Small number to prevent division by zero

        mse_error_percent = (mse_error / (actual_nonzero ** 2)) * 100
        mae_error_percent = (mae_error / abs(actual_nonzero)) * 100

        mse_sum_error += mse_error_percent
        mae_sum_error += mae_error_percent
        sum_actual_price += abs(actual)  # Use abs to ensure positive values for averaging
        count += 1

    # Calculate average errors
    mse_avg_error = mse_sum_error / count
    mae_avg_error = mae_sum_error / count
    arpe_avg_error = (mae_sum_error / sum_actual_price) * 100  # Convert to percentage

    # Print the error report
    print("-" * 100)
    print(f"Average Mean Square Error - MSE (%): {mse_avg_error:.2f}")
    print(f"Average Mean Absolute Percentage Error - MAE (%): {mae_avg_error:.2f}")
    print(f"Average Relative Percentage Error - ARPE (%): {arpe_avg_error:.2f}")
    print("-" * 100)

    return mse_avg_error, mae_avg_error, arpe_avg_error

error_metrics = generate_error_report_from_table(comparison_table)

----------------------------------------------------------------------------------------------------
Average Mean Square Error - MSE (%): 0.58
Average Mean Absolute Percentage Error - MAE (%): 6.45
Average Relative Percentage Error - ARPE (%): 0.06
----------------------------------------------------------------------------------------------------


# Here we are going to predict the parameter and not the price 

In [231]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, RegressorMixin
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam
from scipy.stats import norm

## Heston Pricing Fonction

In [232]:
from scipy.integrate import quad

# Define the characteristic function for the Heston model
def heston_characteristic_function(u, S0, K, r, T, kappa, theta, sigma, rho, v0):
    xi = kappa - rho * sigma * 1j * u
    d = np.sqrt((rho * sigma * 1j * u - xi)**2 + sigma**2 * (-u * 1j - u**2))
    g = (xi - d) / (xi + d)
    C = r * 1j * u * T + (kappa * theta) / sigma**2 * ((xi - d) * T - 2 * np.log((1 - g * np.exp(-d * T)) / (1 - g)))
    D = (xi - d) / sigma**2 * ((1 - np.exp(-d * T)) / (1 - g * np.exp(-d * T)))
    return np.exp(C + D * v0 + 1j * u * np.log(S0))

# Function to compute call and put option prices using the Heston model
def heston_option_price(S0, K, r, T, kappa, theta, sigma, rho, v0, option_type):
    if option_type == 1:  # Call option
        integrand = lambda u: np.real(np.exp(-1j * u * np.log(K)) / (1j * u) * heston_characteristic_function(u - 1j, S0, K, r, T, kappa, theta, sigma, rho, v0))
        integral, _ = quad(integrand, 0, np.inf)
        return np.exp(-r * T) * (S0 * 0.5 - np.exp(-r * T) / np.pi * integral)
    else:  # Put option
        integrand = lambda u: np.real(np.exp(-1j * u * np.log(K)) / (1j * u) * heston_characteristic_function(u - 1j, S0, K, r, T, kappa, theta, sigma, rho, v0))
        integral, _ = quad(integrand, 0, np.inf)
        return np.exp(-r * T) / np.pi * integral - S0 + K * np.exp(-r * T)


In [233]:
# Custom loss function that computes the MSE between observed and model-based prices
def custom_loss(y_true, params_pred, S, K, T, r):
    kappa, theta, sigma, rho, v0 = params_pred[:,0], params_pred[:,1], params_pred[:,2], params_pred[:,3], params_pred[:,4]
    prices_pred = heston_option_price(S, K, T, r, kappa, theta, sigma, rho, v0)
    return tf.reduce_mean(tf.square(y_true - prices_pred))

In [234]:
# Keras Regressor wrapper to include Heston model in fitting process
class KerasHestonRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, build_fn, epochs=100, batch_size=10, verbose=0):
        self.build_fn = build_fn
        self.epochs = epochs
        self.batch_size = batch_size
        self.verbose = verbose
        self.model = None

    def fit(self, X, y, **kwargs):
        self.model = self.build_fn(input_dim=X.shape[1])
        self.model.fit(X, y, epochs=self.epochs, batch_size=self.batch_size, verbose=self.verbose, **kwargs)
        return self

    def predict(self, X, **kwargs):
        return self.model.predict(X, **kwargs)

In [235]:
# Define the model building function
def build_heston_parameter_model(input_dim):
    model = Sequential([
        Dense(128, input_shape=(input_dim,)),
        BatchNormalization(),
        Activation('relu'),
        Dropout(0.3),
        
        Dense(128),
        BatchNormalization(),
        Activation('relu'),
        Dropout(0.3),
        
        Dense(64),
        BatchNormalization(),
        Activation('relu'),
        Dropout(0.2),

        Dense(32),
        BatchNormalization(),
        Activation('relu'),
        Dropout(0.1),

        Dense(5)  # Outputs five Heston parameters: kappa, theta, sigma, rho, v0
    ])
    
    model.compile(optimizer=legacy_optimizers.Adam(learning_rate=0.001), loss='mean_squared_error')
    return model


In [236]:
# Define preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['index_price', 'mark_iv', 'year_fraction', 'strike', 'rf_rate']),
        ('cat', OneHotEncoder(), ['option_type'])
    ])

# Splitting data
features = df.drop('price', axis=1)
target = df['price']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Define the full pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', KerasRegressor(build_fn=build_heston_parameter_model, epochs=2, verbose=1))  # Note the epochs and verbose level
])

# Now fit the pipeline, not just the model
pipeline.fit(X_train, y_train)

Epoch 1/2
14720/14720 [==============================] - 143s 10ms/step - loss: 38358644.0000
Epoch 2/2
14720/14720 [==============================] - 141s 10ms/step - loss: 5140185.5000


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['index_price', 'mark_iv',
                                                   'year_fraction', 'strike',
                                                   'rf_rate']),
                                                 ('cat', OneHotEncoder(),
                                                  ['option_type'])])),
                ('regressor',
                 KerasRegressor(build_fn=<function build_heston_parameter_model at 0x32481cea0>,
                                epochs=2, verbose=1))])

In [237]:
def calculate_prices_from_params(X_test, params_pred):
    # Extract relevant features for pricing from X_test
    S = X_test[:, 0]  # assuming index_price is the first column
    K = X_test[:, 1]  # assuming strike is the second column
    T = X_test[:, 2]  # assuming year_fraction is the third column
    r = X_test[:, 3]  # assuming rf_rate is the fourth column
    option_type = X_test[:, 4]  # assuming option_type is the fifth column (0 for put, 1 for call)

    prices = []
    for i in range(len(params_pred)):
        kappa, theta, sigma, rho, v0 = params_pred[i]
        if option_type[i] == 1:  # Call
            price = heston_call_price(S[i], K[i], r[i], T[i], kappa, theta, sigma, rho, v0)
        else:  # Put
            price = heston_put_price(S[i], K[i], r[i], T[i], kappa, theta, sigma, rho, v0)
        prices.append(price)
    
    return np.array(prices)

# Predict parameters
params_pred = pipeline.predict(X_test)

# Calculate option prices from predicted parameters
predicted_prices = calculate_prices_from_params(X_test, params_pred)

# Now evaluate using these prices
r2 = r2_score(y_test, predicted_prices)
rmse = np.sqrt(mean_squared_error(y_test, predicted_prices))

print(f"R^2 Score: {r2:.2f}")
print(f"RMSE: {rmse:.2f}")


1150/1150 [==============================] - 3s 3ms/step


InvalidIndexError: (slice(None, None, None), 0)